In [5]:
pip install transformers accelerate bitsandbytes torch

In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import time

# Model Adı (Daha küçük bir LLM seçildi, büyük modeller için VRAM'a dikkat edin)
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
# GPU kullanılabiliyorsa 'cuda', değilse 'cpu'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## 1. FP32 (Normal) Model Yükleme ve Bellek Kullanımını Kontrol Etme
print(f"--- 1. Normal (FP32) Model Yükleniyor ---")
try:
    # Modelin FP32 ağırlıklarıyla yükleneceği varsayılır (Bellekte yer kaplar)
    model_fp32 = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.float32,
        low_cpu_mem_usage=True
    ).to(device)

    # Bellek kontrolü (Sadece GPU'daysa anlamlıdır)
    if device == 'cuda':
        fp32_mem = torch.cuda.memory_allocated(device) / (1024**3)
        print(f"FP32 Model GPU Bellek Kullanımı: {fp32_mem:.2f} GB")
        del model_fp32
        torch.cuda.empty_cache()

except Exception as e:
    print(f"FP32 model yüklenirken hata oluştu (normaldir eğer yetersiz VRAM varsa): {e}")


# --- 2. Kantizasyon Ayarlarını Tanımlama (4-bit/INT4) ---
# BitsAndBytesConfig ile 4-bit kantizasyon ayarları yapılır.
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                   # Ağırlıkları 4-bit tamsayı (INT4) olarak yükle
    bnb_4bit_quant_type="nf4",           # 4-bit NormalFloat formatını kullan (genellikle daha iyi performans)
    bnb_4bit_use_double_quant=True,      # Çift kantizasyon (ek bellek tasarrufu)
    bnb_4bit_compute_dtype=torch.bfloat16 # Hesaplamaları bfloat16'da yap (doğruluk için)
)

print("\n--- 2. Kantize Edilmiş (INT4) Model Yükleniyor ---")

start_time = time.time()
# Modeli kantizasyon ayarlarıyla yükleme
model_quantized = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto" # Mümkün olan en verimli şekilde GPU/CPU'ya dağıt
)
end_time = time.time()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

print(f"INT4 Model Yükleme Süresi: {end_time - start_time:.2f} saniye")

## 3. Kantize Modelin Bellek Kullanımını Kontrol Etme
if device == 'cuda':
    quant_mem = torch.cuda.memory_allocated(device) / (1024**3)
    print(f"INT4 Model GPU Bellek Kullanımı: {quant_mem:.2f} GB")
    # FP32 belleğe göre çok daha küçük bir değer görmelisiniz.
    print(f"Bellek Tasarrufu Oranı (yaklaşık): {fp32_mem / quant_mem:.2f} kat")

# --- 4. Kantize Model ile Çıkarım (Inference) Yapma ---
prompt = "Kantizasyon nedir ve neden önemlidir? Cevap: "
inputs = tokenizer(prompt, return_tensors="pt").to(device)

print("\n--- 4. Kantize Model ile Çıkarım Yapılıyor ---")
output_start_time = time.time()
with torch.no_grad():
    outputs = model_quantized.generate(**inputs, max_new_tokens=100)
output_end_time = time.time()

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"Çıkarım Süresi: {output_end_time - output_start_time:.2f} saniye")
print("\nModel Cevabı:")
print("--------------------------------------------------")
print(response)
print("--------------------------------------------------")

--- 1. Normal (FP32) Model Yükleniyor ---
FP32 Model GPU Bellek Kullanımı: 4.83 GB

--- 2. Kantize Edilmiş (INT4) Model Yükleniyor ---
INT4 Model Yükleme Süresi: 3.59 saniye
INT4 Model GPU Bellek Kullanımı: 1.45 GB
Bellek Tasarrufu Oranı (yaklaşık): 3.34 kat

--- 4. Kantize Model ile Çıkarım Yapılıyor ---
Çıkarım Süresi: 12.16 saniye

Model Cevabı:
--------------------------------------------------
Kantizasyon nedir ve neden önemlidir? Cevap: İlk olarak, Kantizasyon, İngilizce dili ile ilgili olarak, İngilizce dili ile ilgili olarak, İngilizce dili ile ilgili olarak, İngilizce dili ile ilgili olarak, İngilizce dili ile ilgili olarak, İng
--------------------------------------------------


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# -----------------------------------------------------
# 1. Modellerin Tanımlanması (Teacher & Student)
# -----------------------------------------------------

# Giriş boyutu (örneğin, bir resimdeki özellik vektörü)
INPUT_SIZE = 784
# Çıkış boyutu (sınıf sayısı)
NUM_CLASSES = 10
# Öğrenci modelin gizli katman boyutu (küçük)
STUDENT_HIDDEN = 64
# Öğretmen modelin gizli katman boyutu (büyük)
TEACHER_HIDDEN = 256

class TeacherModel(nn.Module):
    """Büyük, iyi eğitilmiş model (Daha yüksek kapasite)"""
    def __init__(self):
        super(TeacherModel, self).__init__()
        self.fc1 = nn.Linear(INPUT_SIZE, TEACHER_HIDDEN)
        self.fc2 = nn.Linear(TEACHER_HIDDEN, NUM_CLASSES)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        return self.fc2(x)

class StudentModel(nn.Module):
    """Küçük, hızlı model (Daha düşük kapasite)"""
    def __init__(self):
        super(StudentModel, self).__init__()
        self.fc1 = nn.Linear(INPUT_SIZE, STUDENT_HIDDEN)
        self.fc2 = nn.Linear(STUDENT_HIDDEN, NUM_CLASSES)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        return self.fc2(x)

# -----------------------------------------------------
# 2. Distillation Kayıp Fonksiyonu (Daha önceki yanıttan)
# -----------------------------------------------------

class DistillationLoss(nn.Module):
    def __init__(self, temperature=4.0, alpha=0.7):
        super(DistillationLoss, self).__init__()
        self.T = temperature
        self.alpha = alpha
        self.hard_criterion = nn.CrossEntropyLoss()
        self.kl_div = nn.KLDivLoss(reduction='batchmean')

    def forward(self, student_logits, labels, teacher_logits):
        # 1. Sert Etiket Kaybı
        loss_hard = self.hard_criterion(student_logits, labels)

        # 2. Yumuşak Etiket Kaybı (KL Divergence)
        p_t = F.softmax(teacher_logits / self.T, dim=1)
        log_p_s = F.log_softmax(student_logits / self.T, dim=1)
        loss_soft = self.kl_div(log_p_s, p_t) * (self.T * self.T)

        # 3. Toplam Distillation Kaybı
        loss = self.alpha * loss_hard + (1 - self.alpha) * loss_soft

        return loss, loss_hard.item(), loss_soft.item()

# -----------------------------------------------------
# 3. Öğrenci Eğitim Döngüsü
# -----------------------------------------------------

# Model ve Parametrelerin Başlatılması
teacher_model = TeacherModel()
student_model = StudentModel()

# **Önemli Not:** Gerçek bir senaryoda, TeacherModel önceden eğitilmiş ve sabitlenmiştir!
# Burada, örnek için sadece StudentModel eğitilir.
teacher_model.eval() # Öğretmen modeli değerlendirme moduna ayarla (Eğitilmeyecek)

kd_loss_fn = DistillationLoss(temperature=5.0, alpha=0.6)
optimizer = optim.Adam(student_model.parameters(), lr=0.001)

# Eğitim Parametreleri
NUM_EPOCHS = 5
BATCH_SIZE = 64
NUM_BATCHES = 20 # Veri kümesi simülasyonu

print(f"Öğrenci Gizli Boyut: {STUDENT_HIDDEN}, Öğretmen Gizli Boyut: {TEACHER_HIDDEN}")
print(f"Distillation Sıcaklığı (T): {kd_loss_fn.T}, Alpha (α): {kd_loss_fn.alpha}\n")

student_model.train() # Öğrenci modeli eğitim moduna ayarla

for epoch in range(NUM_EPOCHS):
    epoch_loss = 0.0

    # Veri Kümesi Simülasyonu
    for batch_idx in range(NUM_BATCHES):

        # Rastgele Veri Üretimi
        inputs = torch.randn(BATCH_SIZE, INPUT_SIZE)
        labels = torch.randint(0, NUM_CLASSES, (BATCH_SIZE,))

        # Adım 1: Öğretmen Çıktısını Al (Sabitlenmiş)
        with torch.no_grad(): # Öğretmen modelin gradyanları hesaplanmaz
            teacher_logits = teacher_model(inputs)

        # Adım 2: Öğrenci Çıktısını Al
        student_logits = student_model(inputs)

        # Adım 3: Kaybı Hesapla (KD Loss)
        total_loss, hard_loss, soft_loss = kd_loss_fn(
            student_logits,
            labels,
            teacher_logits
        )

        # Adım 4: Geri Yayılım ve Optimizasyon
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        epoch_loss += total_loss.item()

    avg_loss = epoch_loss / NUM_BATCHES
    print(f"Epoch {epoch+1}/{NUM_EPOCHS} | Ortalama KD Kaybı: {avg_loss:.4f} | Son Soft Loss: {soft_loss:.4f} | Son Hard Loss: {hard_loss:.4f}")

print("\nEğitim tamamlandı. Öğrenci modeli artık öğretmen bilgisi ile eğitilmiştir.")

Öğrenci Gizli Boyut: 64, Öğretmen Gizli Boyut: 256
Distillation Sıcaklığı (T): 5.0, Alpha (α): 0.6

Epoch 1/5 | Ortalama KD Kaybı: 1.4181 | Son Soft Loss: 0.0425 | Son Hard Loss: 2.3245
Epoch 2/5 | Ortalama KD Kaybı: 1.4175 | Son Soft Loss: 0.0462 | Son Hard Loss: 2.3198
Epoch 3/5 | Ortalama KD Kaybı: 1.4114 | Son Soft Loss: 0.0421 | Son Hard Loss: 2.2894
Epoch 4/5 | Ortalama KD Kaybı: 1.4109 | Son Soft Loss: 0.0382 | Son Hard Loss: 2.3267
Epoch 5/5 | Ortalama KD Kaybı: 1.4146 | Son Soft Loss: 0.0384 | Son Hard Loss: 2.3247

Eğitim tamamlandı. Öğrenci modeli artık öğretmen bilgisi ile eğitilmiştir.


In [8]:
import torch
import torch.nn as nn
import torch.nn.utils.prune as prune

# --- 1. Model ve Ağırlık Hazırlığı ---
# Basit bir doğrusal katman (giriş 50, çıkış 10)
layer = nn.Linear(50, 10)

# Ağırlıkları rastgele başlatıyoruz
# Ağırlık matrisi boyutu: [10, 50] = 500 parametre
print(f"Başlangıç ağırlık boyutu: {layer.weight.shape}")
print("-" * 30)


# --- 2. Başlangıç Seyreklik Kontrolü ---
def get_sparsity(model):
    """Modeldeki toplam sıfır ağırlıkların oranını hesaplar."""
    total_params = 0
    zero_params = 0

    # Sadece 'weight' ve 'bias' parametrelerini kontrol et
    for name, module in model.named_modules():
        if isinstance(module, nn.Linear) or isinstance(module, nn.Conv2d):
            # PyTorch'un budanmış modelleri, 'weight_orig' ve 'weight_mask' kullanır
            weight = getattr(module, 'weight', None)

            if weight is not None:
                total_params += weight.numel()
                zero_params += torch.sum(weight == 0).item()

    if total_params == 0:
        return 0.0

    return (zero_params / total_params) * 100

initial_sparsity = get_sparsity(layer)
print(f"Budama öncesi seyreklik oranı: {initial_sparsity:.2f}%")
print("-" * 30)

# --- 3. Yapılandırılmamış Budama Uygulaması ---

# Bu işlem, L1 normuna (mutlak değer) göre en düşük
# %30'luk kısmı budar (maskeyi sıfır yapar).
# 'name': budanacak parametre, 'amount': budama oranı
prune.l1_unstructured(layer, name='weight', amount=0.3)

print("✅ Budama işlemi tamamlandı (L1 Unstructured, %30)")
print("-" * 30)

# --- 4. Budama Sonrası Kontrol ---

# Budama sonrasında katmanda iki yeni parametre oluşur:
# 1. 'weight_orig': Orijinal ağırlık değerleri
# 2. 'weight_mask': Budamayı uygulayan binary (0 veya 1) maske
print(f"Katman parametreleri: {list(layer.named_parameters())}")

# PyTorch, ileri yayılım (forward pass) sırasında 'weight_orig' ve 'weight_mask'
# kullanarak dinamik olarak budanmış ağırlığı ('weight') oluşturur.
# Oluşturulan 'weight' maske ile sıfırları içerir:
final_sparsity = get_sparsity(layer)
print(f"Budama sonrası seyreklik oranı: {final_sparsity:.2f}%")
print("-" * 30)

# --- 5. Budamayı Kalıcı Hale Getirme (Gerektiğinde) ---

# Modelin daha sonra eğitilmesi veya dışa aktarılması (export) için
# maske ve orijinal ağırlık değerlerini birleştirip kalıcı hale getirmeliyiz.
# Bu işlem, orijinal ağırlıkları ve maskeyi kaldırarak **sıfırları**
# doğrudan 'weight' parametresine yazar ve budama araçlarını temizler.
# **DİKKAT:** Bu adımdan sonra budama geri alınamaz!

# prune.remove(layer, 'weight')
# print("Budama kalıcı hale getirildi ve budama yardımcıları temizlendi.")
# print(f"Kalıcı hale getirilmiş parametreler: {list(layer.named_parameters())}")
# print("-" * 30)

# Budamadan sonra ağırlık boyutu hala [10, 50] olacaktır, ancak içindeki
# değerlerin yaklaşık %30'u **tam olarak sıfırdır**.
# Modelin bellekte kapladığı yer veya hesaplama (FLOPs) ancak bu model
# seyrekleştirmeyi destekleyen bir formata dönüştürülüp (örneğin ONNX'te)
# özel bir donanım/kütüphane üzerinde çalıştırılırsa azalır.

Başlangıç ağırlık boyutu: torch.Size([10, 50])
------------------------------
Budama öncesi seyreklik oranı: 0.00%
------------------------------
✅ Budama işlemi tamamlandı (L1 Unstructured, %30)
------------------------------
Katman parametreleri: [('bias', Parameter containing:
tensor([-0.1332,  0.0016, -0.0939,  0.0789,  0.1242, -0.0854,  0.0027, -0.0797,
        -0.1292, -0.0543], requires_grad=True)), ('weight_orig', Parameter containing:
tensor([[-1.2679e-01,  1.2981e-01, -3.8986e-02, -1.3693e-01, -2.6731e-03,
         -8.0254e-02, -7.1579e-02,  7.7183e-02, -1.0101e-03,  9.6338e-02,
         -1.3199e-02, -2.3079e-02, -5.1178e-02, -8.1450e-02, -2.2775e-02,
         -6.3556e-02, -8.9668e-02, -8.3085e-02,  8.9932e-02,  4.0637e-03,
         -7.7488e-02,  1.0692e-01,  1.2273e-01, -1.3595e-01,  6.2558e-02,
          3.5471e-02,  5.8022e-02, -9.7679e-02, -1.3083e-01,  1.0553e-01,
         -2.4606e-03,  1.0615e-01, -1.2171e-01,  1.0275e-01,  7.0323e-02,
         -2.5839e-03,  1.1090e-01,

Bu ilk kod hücresi, gerekli kütüphaneleri (transformers, accelerate, bitsandbytes, torch) yüklemek için kullanılır. Bu kütüphaneler genellikle büyük dil modellerini (LLM) yüklemek, hızlandırmak ve nicelleştirmek (quantization) için gereklidir.

Bu kod hücresi, büyük dil modellerini (LLM) nicelleştirmenin (quantization) belleğe ve çıkarım hızına olan etkisini gösterir.

### **Adım 1: Normal (FP32) Model Yükleme ve Bellek Kullanımını Kontrol Etme**

*   Model, tam hassasiyetli (FP32) ağırlıklarla yüklenmeye çalışılır. Bu, genellikle daha fazla bellek gerektirir.
*   Eğer GPU varsa, FP32 modelin ne kadar GPU belleği kullandığı hesaplanır. Yetersiz VRAM varsa bu adım hata verebilir, bu normaldir ve nicelleştirmenin neden gerekli olduğunu gösterir.

### **Adım 2: Kantizasyon Ayarlarını Tanımlama (4-bit/INT4)**

*   `BitsAndBytesConfig` kullanarak 4-bit nicelleştirme ayarları yapılır:
    *   `load_in_4bit=True`: Model ağırlıklarını 4-bit tamsayı (INT4) olarak yükler.
    *   `bnb_4bit_quant_type="nf4"`: 4-bit NormalFloat formatını kullanır, genellikle daha iyi performans sağlar.
    *   `bnb_4bit_use_double_quant=True`: Çift nicelleştirme kullanarak ek bellek tasarrufu sağlar.
    *   `bnb_4bit_compute_dtype=torch.bfloat16`: Hesaplamaları bfloat16 (karma hassasiyet) formatında yapar, bu doğruluk ve hız arasında iyi bir denge sağlar.

*   Model, tanımlanan `BitsAndBytesConfig` ayarlarıyla nicelleştirilmiş olarak yüklenir. `device_map="auto"` parametresi, modelin parçalarını otomatik olarak GPU/CPU arasında dağıtarak en verimli şekilde yüklenmesini sağlar.
*   Modelin yüklenme süresi ölçülür ve yazdırılır.

### **Adım 3: Kantize Modelin Bellek Kullanımını Kontrol Etme**

*   Eğer GPU varsa, nicelleştirilmiş modelin GPU bellek kullanımı hesaplanır. Bu değerin FP32 modele göre çok daha düşük olması beklenir.
*   Bellek tasarrufu oranı (FP32 bellek / INT4 bellek) hesaplanarak nicelleştirmenin ne kadar etkili olduğu gösterilir.

### **Adım 4: Kantize Model ile Çıkarım (Inference) Yapma**

*   Önceden tanımlanmış bir `prompt` (istek) kullanılarak nicelleştirilmiş modelden bir yanıt üretilir.
*   Çıkarım (inference) süresi ölçülür ve modelin cevabı ekrana yazdırılır.

**Genel Amaç:** Bu kod, özellikle kısıtlı bellek kaynaklarına sahip cihazlarda (örneğin, küçük GPU'lar veya CPU'lar) büyük dil modellerini çalıştırmak için nicelleştirmenin (quantization) ne kadar faydalı olduğunu göstermektedir. Nicelleştirme, model boyutunu ve bellek kullanımını önemli ölçüde azaltırken, kabul edilebilir bir doğruluk seviyesi ile çıkarım hızını artırabilir.

Bu kod hücresi, makine öğreniminde **Bilgi Distilasyonu (Knowledge Distillation)** tekniğini uygulamak için bir örnek sunar. Bu teknik, büyük ve karmaşık bir "Öğretmen Modelden" (Teacher Model) öğrenerek daha küçük, daha hızlı ve daha verimli bir "Öğrenci Model" (Student Model) eğitmeyi amaçlar.

### **1. Modellerin Tanımlanması (Teacher & Student)**

*   **`INPUT_SIZE`**, **`NUM_CLASSES`**, **`STUDENT_HIDDEN`**, **`TEACHER_HIDDEN`** gibi sabitler tanımlanır. Bu, modelin giriş/çıkış boyutlarını ve gizli katmanların büyüklüğünü belirler.
*   **`TeacherModel`** sınıfı, daha büyük bir gizli katmana (örneğin, `TEACHER_HIDDEN = 256`) sahip basit bir tam bağlı (fully connected) sinir ağı olarak tanımlanır. Bu, kapasitesi yüksek, iyi eğitilmiş bir modelin simülasyonudur.
*   **`StudentModel`** sınıfı, daha küçük bir gizli katmana (örneğin, `STUDENT_HIDDEN = 64`) sahip benzer bir mimariye sahiptir. Bu, Öğretmen Modelden bilgi öğrenerek daha verimli hale gelecek olan modeldir.

### **2. Distillation Kayıp Fonksiyonu (DistillationLoss)**

*   **`DistillationLoss`** sınıfı, bilgi distilasyonu için özel bir kayıp fonksiyonu uygular. İki ana bileşeni vardır:
    *   **`hard_criterion` (Sert Etiket Kaybı):** Öğrenci modelinin gerçek etiketlere (ground truth) göre ne kadar iyi performans gösterdiğini ölçen standart bir `CrossEntropyLoss`'tur. Bu, öğrencinin temel görevi öğrenmesini sağlar.
    *   **`kl_div` (Yumuşak Etiket Kaybı / KL Divergence):** Öğrenci modelinin çıktılarının (logit'lerinin), öğretmen modelinin "yumuşak hedefleri"ne (softmax çıkışları) ne kadar benzediğini ölçer. `temperature` (sıcaklık) parametresi, öğretmen çıktılarının yumuşaklığını kontrol eder ve öğrencinin daha zengin bilgi öğrenmesine yardımcı olur.
*   **`alpha`** parametresi, sert etiket kaybı ile yumuşak etiket kaybı arasındaki dengeyi ayarlar.
*   `forward` metodu, her iki kayıp bileşenini hesaplar ve bunları `alpha` değerine göre ağırlıklandırarak toplam distilasyon kaybını döndürür.

### **3. Öğrenci Eğitim Döngüsü**

*   **`TeacherModel`** ve **`StudentModel`** örnekleri oluşturulur.
*   **Önemli Not:** Gerçek bir senaryoda, Öğretmen Model genellikle önceden eğitilmiş ve ağırlıkları dondurulmuştur (`teacher_model.eval()` ve `with torch.no_grad()`). Bu örnekte de `teacher_model.eval()` ile değerlendirme moduna alınmış ve `torch.no_grad()` ile gradyan hesaplamaları devre dışı bırakılarak sabitlenmiş gibi davranılmıştır.
*   **`DistillationLoss`** için sıcaklık (`temperature`) ve alfa (`alpha`) değerleri belirlenir.
*   **`optimizer`**, sadece öğrenci modelinin parametrelerini güncelleyecek şekilde tanımlanır.
*   Belirlenen `NUM_EPOCHS` (dönem) boyunca eğitim döngüsü çalışır:
    *   Her dönemde, rastgele girişler (`inputs`) ve etiketler (`labels`) oluşturularak bir veri kümesi simüle edilir.
    *   **Öğretmen çıktısı alınır:** `teacher_model` kullanılarak girişler üzerinde `teacher_logits` elde edilir. `torch.no_grad()` bloğu sayesinde bu adımda öğretmen modelinin ağırlıkları güncellenmez.
    *   **Öğrenci çıktısı alınır:** `student_model` kullanılarak girişler üzerinde `student_logits` elde edilir.
    *   **Distilasyon kaybı hesaplanır:** `kd_loss_fn` kullanılarak `student_logits`, `labels` ve `teacher_logits` ile toplam kayıp (`total_loss`), sert kayıp (`hard_loss`) ve yumuşak kayıp (`soft_loss`) hesaplanır.
    *   **Geri yayılım ve optimizasyon:** `total_loss` geri yayılır ve `optimizer.step()` ile öğrenci modelinin ağırlıkları güncellenir.
*   Her dönemin sonunda ortalama kayıp değerleri yazdırılır.

**Genel Amaç:** Bu kod, bir öğrenci modelin, daha güçlü bir öğretmen modelden nasıl bilgi aktarımı yapabileceğini ve bu sayede kendi başına (direkt olarak etiketlerden öğrenmek yerine) daha iyi performans gösterebileceğini simüle eder. Bu, model sıkıştırma ve hızlandırma tekniklerinden biridir.

Bu kod hücresi, derin öğrenme modellerinde **ağırlık budama (weight pruning)** tekniğini görselleştirmek için PyTorch'un `torch.nn.utils.prune` modülünü kullanır. Budama, modeldeki önemsiz ağırlıkları kaldırarak modelin boyutunu ve hesaplama yükünü azaltmayı hedefler.

### **1. Model ve Ağırlık Hazırlığı**

*   Basit bir doğrusal katman (`nn.Linear(50, 10)`) oluşturulur. Bu katman 50 giriş özelliğini 10 çıkış özelliğine eşler ve 500 ağırlık parametresine sahiptir (10 * 50).
*   Başlangıç ağırlık matrisinin boyutu yazdırılır.

### **2. Başlangıç Seyreklik Kontrolü (`get_sparsity`)**

*   `get_sparsity` adında bir yardımcı fonksiyon tanımlanır. Bu fonksiyon, bir modeldeki toplam sıfır ağırlıkların yüzdesini hesaplar.
*   `named_modules()` ile modeldeki tüm modüller gezilir ve `nn.Linear` veya `nn.Conv2d` gibi katmanlar tespit edilir.
*   `weight.numel()` ile toplam ağırlık sayısı, `torch.sum(weight == 0)` ile de sıfır olan ağırlıkların sayısı bulunur.
*   Budama öncesindeki seyreklik oranı (genellikle %0 veya sıfıra yakın bir değer) yazdırılır.

### **3. Yapılandırılmamış Budama Uygulaması (`prune.l1_unstructured`)**

*   `prune.l1_unstructured` fonksiyonu kullanılarak doğrusal katmanın ağırlıkları budanır.
*   `layer`: Budama yapılacak katman (`nn.Linear` örneği).
*   `name='weight'`: Budama yapılacak parametrenin adı (burada ağırlık matrisi).
*   `amount=0.3`: Ağırlıkların %30'unun budanacağı anlamına gelir. `L1 normuna` (mutlak değer) göre en küçük %30'luk kısım sıfır yapılır.

### **4. Budama Sonrası Kontrol**

*   Budama işlemi tamamlandıktan sonra, PyTorch orijinal `weight` parametresini `weight_orig` ve `weight_mask` olmak üzere iki yeni parametreye dönüştürür:
    *   `weight_orig`: Ağırlıkların orijinal değerlerini tutar.
    *   `weight_mask`: Binary (0 veya 1) bir maske olup, budanan (sıfır olan) ağırlıkların konumunu gösterir.
*   `layer.named_parameters()` çıktısı bu yeni parametreleri gösterir.
*   Modelin ileri yayılımı (forward pass) sırasında, PyTorch otomatik olarak `weight_orig` ve `weight_mask`'i çarparak "gerçek" budanmış `weight` parametresini oluşturur.
*   `get_sparsity` fonksiyonu tekrar çağrılarak budama sonrası seyreklik oranı hesaplanır. Bu değerin yaklaşık olarak %30 olması beklenir.

### **5. Budamayı Kalıcı Hale Getirme (Gerektiğinde - Yorum Satırında)**

*   `prune.remove(layer, 'weight')` komutu (şu anda yorum satırında), budama maskesini ve orijinal ağırlıklarını birleştirerek kalıcı hale getirir.
*   Bu işlem, `weight_orig` ve `weight_mask`'i kaldırır ve ağırlık matrisindeki sıfırları doğrudan `weight` parametresine yazar.
*   **Uyarı:** Bu adım geri alınamaz ve modelin ağırlıklarını kalıcı olarak değiştirir.

**Genel Amaç:** Bu kod, model budamanın temel mekanizmalarını ve PyTorch'ta nasıl uygulanabileceğini gösterir. Budama, modelin daha az bellek kullanmasını ve daha hızlı çalışmasını sağlayarak dağıtım için daha uygun hale getirilmesine yardımcı olabilir, özellikle kısıtlı kaynaklara sahip cihazlarda.